In [1]:
import os
import pickle
from gensim.models import KeyedVectors
from gensim.models.wrappers import FastText

In [2]:
from tweet.tweet import Tweet
from preprocessing import preprocessing, tree_create

In [3]:
%%time
modelw2v = KeyedVectors.load_word2vec_format("../model/SBW-vectors-300-min5.bin.gz", binary=True)

CPU times: user 1min 11s, sys: 2.06 s, total: 1min 13s
Wall time: 3min 1s


In [ ]:
%%time
modelft = FastText.load_fasttext_format('../model/SENTIMENT_UBA_v4_d100.bin')

In [4]:
ROOT_OUT = '../pickle/'

In [5]:
lexicon_attack = ['falso', 'mentira']

In [12]:
# Lexicon esperado
expected_lexicon = ['mentira', 'mentiroso', 'falso', 'mentirosa', 'verdad', 'verdadero', 'engañoso', 'mentir']

## Pipeline

**Primer resultado**

In [7]:
def tagged(tweets, root, lexicon, model, n):
    lexicons = [lexicon]
    tree = tree_create(tweets, root)
    
    for i in range(n):
        if 0 < i:
            new = lexicons[i-1] + [word for word, d in model.most_similar(positive=lexicons[i-1])]
            lexicons.append(new)
       
        for _, tweet in tweets.items():
            
            if tweet.stance == 'neutral':
                tweet.is_attack(lexicons[i])
            
            if tweet.stance == 'neutral':
                tweet.is_support()
        
        name = root + 'n' + str(i) + '.pkl'
        with open(os.path.join(ROOT_OUT, name), 'wb') as fd:
            pickle.dump(tree, fd)
    
    return lexicons

In [8]:
tweets, root = preprocessing('perfil.status.1191780164549697536.json')
lexicons = tagged(tweets, root, lexicon_attack, modelw2v, 3)

El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente
El elemento no está presente


In [11]:
lexicons

[['falso', 'mentira'],
 ['falso',
  'mentira',
  'falsa',
  'mentirilla',
  'verdad',
  'fraudulencia',
  'engaño',
  'infundio',
  'mentiras',
  'andrómina',
  'patraña',
  'encubriéndola'],
 ['falso',
  'mentira',
  'falsa',
  'mentirilla',
  'verdad',
  'fraudulencia',
  'engaño',
  'infundio',
  'mentiras',
  'andrómina',
  'patraña',
  'encubriéndola',
  'embaucamiento',
  'inauténtica',
  'mangancia',
  'autocondena',
  'extrañándose',
  'tacharme',
  'trola',
  'fechorias',
  'trocaré',
  'desverguenza']]

In [ ]:
# UNA IDEA PRIMERO CALCULAR LOS LEXICONES Y DESPUES TAGGEAR

IDEA FRANCO:

1- AGRANDO LEXICON A PARTIR DEL MODELO DE CRISTIAN

2- CALCULO UN UMBRAL DEL MODELO DE CRISTIAN

3- PARA CADA (PALABRAL, TOKENT) DEL LEXICON X TWEET, SI DISTANCIA(PALABRAL, TOKENT) > UMBRAL ENTONCES LEXICON + TOKENT 

In [ ]:
from itertools import product

In [ ]:
def taggedcross(modelft, tweets, root, lexicon, u, n):
    
    lexicons = [lexicon]
    #tree = tree_create(tweets, root)
    for i in range(n):
        new = lexicons[i]
        for _, tweet in tweets.items():
            
            merge = product(lexicons[i], tweet.tokens)

            for wordl, token in list(merge):
                if wordl and token in modelft.wv.vocab:
                    if modelft.similarity(wordl, token) >= u:
                        if token not in lexicon_attack:
                            new.append(token)
        
        lexicons.append(new)
    return lexicons

In [ ]:
tweets, root = preprocessing('perfil.status.1191780164549697536.json')

In [ ]:
lexicons = taggedcross(modelft, tweets, root, lexicon_attack, 0.65, 3)

In [ ]:
len(lexicons[2])

In [ ]:
w = filter(lambda x: x in modelw2v.vocab, lexicon_attack)
lexicon_attack += [word for word, d in modelw2v.most_similar(positive=w)]

In [ ]:
lexicon_attack

In [ ]:
import nltk.translate.bleu_score as bleu
print('BLEU score: {}'.format(bleu.sentence_bleu(tree.replies[19].tokens, tree.tokens)))

**API TWITTER**

In [ ]:
CONSUMER_KEY = 'tRe3ZBhnXDPBEKyTi4ifvPTC7' 
CONSUMER_SECRET = '0dmQlSimyEmaOPiDoFZTF0wYjO0LSzSdWmCP7afZkNHZF94Af5'
ACCESS_TOKEN = '421837149-YVewRWLSu8xyYNU1H4NtELUejGB8lzvT4KCfallG'
ACCESS_SECRET = 'mFsKYpvJaWvZGHRoy874Z4IF09CRP69srlVj2yjN6mH5L'